<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/satd_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instala as dependências

In [1]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
All depenpencies installed!
Details in install.log


# 2. Importa as dependências, configurações e classes de apoio

In [46]:
from pydriller import Repository
from datetime import datetime
import tqdm
import os
import re

from jira import JIRA
import os
from datetime import datetime, timedelta
import tqdm
import sqlite3
import subprocess
import sqlite3

In [47]:
url_to_repository = 'https://github.com/apache/cassandra.git'
path_to_repository = 'cassandra'
os.environ['MY_REPOSITORY'] = url_to_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
DATABASE_NAME = "issues_db.db"

os.environ['DATABASE_NAME'] = DATABASE_NAME
# Credentials
os.environ['USERNAME'] = 'armandossrecife'
os.environ['PASSWORD'] = 'sky1979#ce'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

In [31]:
def is_java_comment(line):
    # Regular expression pattern to match Java comments
    comment_pattern = r'^\s*//|^\s*/\*|^\s*\*|^\s*\*/'

    # Use the re.match function to check if the line matches the comment pattern
    return bool(re.match(comment_pattern, line))

In [33]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments

  def get_comments(self) -> dict:
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

class IssuesDatabase:
    def __init__(self, database_name):
        self.database_name = database_name
        self.create_tables()

    def create_tables(self):
        self.conn = sqlite3.connect(self.database_name)
        self.cursor = self.conn.cursor()

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Issues (
                id INTEGER PRIMARY KEY,
                project TEXT,
                key TEXT,
                summary TEXT,
                issue_type TEXT,
                status TEXT,
                priority TEXT,
                description TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Comments (
                id INTEGER PRIMARY KEY,
                key TEXT,
                comment TEXT
            )
        ''')

        self.conn.commit()

    def insert_in_table_issues(self, project, key, summary, issue_type, status, priority, description):
        values = (None, project, key, summary, issue_type, status, priority, description)
        self.cursor.execute('''
            INSERT INTO Issues
            (id, project, key, summary, issue_type, status, priority, description)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', values)

        self.conn.commit()

    def insert_in_table_comments(self, key, comment):
        values = (None, key, comment)
        self.cursor.execute('''
            INSERT INTO Comments
            (id, key, comment)
            VALUES (?, ?, ?)
        ''', values)

        self.conn.commit()

    def show_content(self, table):
        query = f"SELECT * FROM {table}"
        self.cursor.execute(query)

        rows = self.cursor.fetchall()
        for row in rows:
            print(row)

    def show_n_lines(self, table, n):
        query = f"SELECT * FROM {table}"
        self.cursor.execute(query)

        rows = self.cursor.fetchall()

        for i, row in enumerate(rows):
            print(row)
            if i == n:
              break

    def close_connection(self):
        self.conn.close()

# 3. Clona o repositório local

In [4]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/cassandra.git
Cloning into 'cassandra'...
remote: Enumerating objects: 400963, done.
remote: Counting objects: 100% (4055/4055), done.
remote: Compressing objects: 100% (1589/1589), done.
remote: Total 400963 (delta 1860), reused 3592 (delta 1736), pack-reused 396908
Receiving objects: 100% (400963/400963), 398.61 MiB | 24.62 MiB/s, done.
Resolving deltas: 100% (233473/233473), done.


# 4. Analisa os commits

In [5]:
dt1 = datetime(2021, 1, 1, 0, 0, 0)
dt2 = datetime(2021, 12, 30, 0, 0, 0)

# Define the set of SATD keywords
satd_keywords = {'TODO', 'FIXME', 'refactor', 'hack', 'workaround', 'technical debt', 'cleanup', 'clean', 'fix'}

# Initialize sets to store commits and issues with SATD
commits_with_satd = set()
issues_with_satd = set()
dict_commit_msg = dict()

# Step 3: Analyze Commits
print('Aguarde...')
my_traverser_commits = Repository(path_to_repository, since=dt1, to=dt2).traverse_commits()
total_commits = len(list(my_traverser_commits))
for commit in tqdm.tqdm(Repository(path_to_repository, since=dt1, to=dt2).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
    for keyword in satd_keywords:
        if keyword in commit.msg:
            commits_with_satd.add(commit.hash)
            dict_commit_msg[commit.hash] = commit.msg

Aguarde...


Progress commit anlysis: 100%|██████████| 1197/1197 [00:00<00:00, 2360.10it/s]


In [48]:
print('Mostra um fragmento do dicionario de commits e msgs')
cont = 0
for k, v in dict_commit_msg.items():
  print(f'k: {k}, v: {v}')
  if cont == 10:
    break
print(dict_commit_msg)

Mostra um fragmento do dicionario de commits e msgs
k: 7214794e3f86b67d96371c78815c1ce03b3a5d6b, v: ninja fix CHANGES.txt
k: 07167c95e3fc6616c313877ccd20eecbb42ef4c0, v: Ninja: fix fqltool tests (CASSANDRA-16384 follow up)
k: 44ca3d322d8444d23d2bd1a5770b62f41de5f0f6, v: ninja fix: CASSANDRA-16261, NoSpamLogger substituted with normal logger at TRACE in MemtableAllocator
k: b04429b57ca8e48f00d0e58e9c76887d4776cd9c, v: ninja-fix debian cassandra.yaml patch for CASSANDRA-14793
k: a7b71eb63eecde97e8ecb70827b09a621040147d, v: ninja-fix CHANGES.txt (CASSANDRA-16477)
k: 07babf3cb10bc11efad48cc7865c7425d7c27ce4, v: Ninja-fix: CASSANDRA-15897 broke one of the CompactStorageUpgrade tests; GOSSIP required
k: eb68380866c9d96592580fefbc1b79a497a674bf, v: Throw IOE in AbstractType.writeValue if value has wrong fixed length
authored by Alexandre Dutra; reviewed by Berenguer Blasi and Andres de la Pena for CASSANDRA-16500
k: 866844743ec3cd23451b0158a7841a026e2e68d6, v: ninja fix CHANGES.txt entry for 

## 4.1 Analisa os diffs dos commits

In [22]:
# Step 4: Analyze Modified Files in Commits
dict_commmit_diffs = dict()
for commit in tqdm.tqdm(Repository(path_to_repository, since=dt1, to=dt2).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
  list_keywords_by_commit = list()
  list_diff_content_by_commit = list()
  for modification in commit.modified_files:
    for line in modification.diff_parsed['added']:
      valor_linha = line[0]
      conteudo_linha = line[1]
      for keyword in satd_keywords:
        if is_java_comment(conteudo_linha):
          if keyword in conteudo_linha:
            commits_with_satd.add(commit.hash)
            list_keywords_by_commit.append(keyword)
            list_diff_content_by_commit.append(conteudo_linha)
  if len(list_keywords_by_commit) > 0:
    elemento = list_keywords_by_commit, list_diff_content_by_commit
    dict_commmit_diffs[commit.hash] = elemento

Progress commit anlysis: 100%|██████████| 1197/1197 [00:16<00:00, 72.73it/s]


In [49]:
cont = 0
for k,v in dict_commmit_diffs.items():
  print(f'k: {k}, v: {v}')
  if cont == 10:
    break

k: 0a1e900a0a042f78d7d5d6625bc98b84eb463e69, v: (['clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean'], ['         * reclaiming too, and it could cause problems to the memtable cleaner algorithm if reclaiming', ' * The cleaner is used by {@link MemtableCleanerThread} in order to reclaim space from memtables, normally', '     * This is a function that schedules a cleaning task, normally flushing of the largest sstable.', '     * the cleaner was able to execute the cleaning operation or if another thread concurrently executed', '     * the same clean operation. If no operation was even attempted, for example because no memtable was', '     * The future will complete with an error if the cleaning operation encounters an error.', '        /** This is incremented when a cleaner is invoked and decremented when a cleaner has completed */', '            // if the cleaning job w

# 5. Analisa os issues do Jira

## 5.1 Configura a instância, projeto e intervalo de datas

In [37]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils('CASSANDRA', jira)

# Define date intervals
print('Define date intervals')
date1 = (2023, 1, 1)
date2 = (2023, 9, 1)
distance = 120

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals


## 5.2 Recupera todos os issues da configuração

In [38]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:  33%|███▎      | 1/3 [00:04<00:09,  4.64s/it]

Range: project=CASSANDRA and created>="2023/01/01" and created<="2023/04/30", qtd issues: 360


Progress Message Analysis:  67%|██████▋   | 2/3 [00:08<00:04,  4.26s/it]

Range: project=CASSANDRA and created>="2023/05/01" and created<="2023/08/28", qtd issues: 309


Progress Message Analysis: 100%|██████████| 3/3 [00:10<00:00,  3.37s/it]

Range: project=CASSANDRA and created>="2023/08/29" and created<="2023/12/26", qtd issues: 96
2023-10-01 19:05:49.522615
Tempo da consulta: 0:00:10.121579


In [39]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 765


# 5.3 Filtra os issues de SATD

In [40]:
# Create an instance of JiraIssues to manage SATD issues
satd_issues = JiraIssues('CASSANDRA', [])

In [42]:
# Iterate through the fetched issues
for issue in all_issues:
    issue_key = issue.key
    issue_summary = issue.fields.summary
    issue_description = issue.fields.description
    issue_comments = [comment.body for comment in issue.fields.comment.comments]

    # Check for SATD keywords in the issue's summary, description, and comments
    if issue_summary is None:
      issue_summary = ""
    if issue_description is None:
      issue_description = ""
    if issue_comments is None:
      issue_comments = ""
    is_satd = any(keyword in (issue_summary + issue_description + ' '.join(issue_comments)) for keyword in satd_keywords)

    if is_satd:
        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        satd_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments)

        # Add the SATD issue to the list of SATD issues
        satd_issues.add_issue(satd_issue)

In [45]:
for each in satd_issues.get_issues():
  print(each)

Key: CASSANDRA-18487, Summary: MappedByteBufferIndexInputProvider can better throw UndeclaredThrowableException, Type: Bug, Status: Resolved
Key: CASSANDRA-18486, Summary: LeveledCompactionStrategy does not check its constructor, Type: Bug, Status: Resolved
Key: CASSANDRA-18485, Summary: CEP-15: (C*) Enhance in-memory FileSystem to work with mmap and support tests to add custom logic, Type: Improvement, Status: Resolved
Key: CASSANDRA-18484, Summary: FunctionCall can throw more specific exceptions, Type: Bug, Status: Open
Key: CASSANDRA-18482, Summary: Test Failure: HintsDisabledTest.testHintedHandoffDisabled, Type: Improvement, Status: Resolved
Key: CASSANDRA-18487, Summary: MappedByteBufferIndexInputProvider can better throw UndeclaredThrowableException, Type: Bug, Status: Resolved
Key: CASSANDRA-18486, Summary: LeveledCompactionStrategy does not check its constructor, Type: Bug, Status: Resolved
Key: CASSANDRA-18485, Summary: CEP-15: (C*) Enhance in-memory FileSystem to work with mm